In [1]:
import os
import pandas as pd
import numpy as np
import nltk
import string
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import *

In [79]:
# nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/mtweed/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
%matplotlib inline

In [2]:
sc = SparkContext("local[*]", "explore")

In [4]:
spark = SparkSession.builder.master('local[*]').appName("explore2").getOrCreate()

In [7]:
blog = spark.read.csv('/spring2021/project1/blogtext.csv', header = True, inferSchema=True)

In [8]:
hack = spark.read.csv('/spring2021/project1/hacker_news_sample.csv', header=True, inferSchema=True)

In [9]:
blog.printSchema()

root
 |-- id: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- topic: string (nullable = true)
 |-- sign: string (nullable = true)
 |-- date: string (nullable = true)
 |-- text: string (nullable = true)



In [10]:
hack.printSchema()

root
 |-- title: string (nullable = true)
 |-- url: string (nullable = true)
 |-- text: string (nullable = true)
 |-- dead: string (nullable = true)
 |-- by: string (nullable = true)
 |-- score: string (nullable = true)
 |-- time: string (nullable = true)
 |-- type: string (nullable = true)
 |-- id: integer (nullable = true)
 |-- parent: integer (nullable = true)
 |-- descendants: integer (nullable = true)
 |-- ranking: timestamp (nullable = true)
 |-- deleted: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)



In [11]:
blogDF = blog.toDF('id','gender','age','topic','sign','date','text')

In [12]:
hackDF = hack.toDF('title','url','text','dead','by','score','time','type','id','parent','descendants','ranking','deleted','timestamp')

In [13]:
blogDF.show()

+-------+------+---+-----------------+--------+------------+--------------------+
|     id|gender|age|            topic|    sign|        date|                text|
+-------+------+---+-----------------+--------+------------+--------------------+
|2059027|  male| 15|          Student|     Leo| 14,May,2004|           Info h...|
|2059027|  male| 15|          Student|     Leo| 13,May,2004|           These ...|
|2059027|  male| 15|          Student|     Leo| 12,May,2004|           In het...|
|2059027|  male| 15|          Student|     Leo| 12,May,2004|           testin...|
|3581210|  male| 33|InvestmentBanking|Aquarius|11,June,2004|             Than...|
|3581210|  male| 33|InvestmentBanking|Aquarius|10,June,2004|             I ha...|
|3581210|  male| 33|InvestmentBanking|Aquarius|10,June,2004|             Some...|
|3581210|  male| 33|InvestmentBanking|Aquarius|10,June,2004|             If a...|
|3581210|  male| 33|InvestmentBanking|Aquarius|10,June,2004|             Take...|
|3581210|  male|

In [14]:
hackDF.show()

+--------------------+--------------------+--------------------+----+---------------+-----+----------+-------+--------+--------+-----------+-------+-------+-------------------+
|               title|                 url|                text|dead|             by|score|      time|   type|      id|  parent|descendants|ranking|deleted|          timestamp|
+--------------------+--------------------+--------------------+----+---------------+-----+----------+-------+--------+--------+-----------+-------+-------+-------------------+
|                null|                null|&gt;<i>which lead...|null|        coldtea| null|1390843873|comment| 7131680| 7127578|       null|   null|   null|2014-01-27 12:31:13|
|                null|                null|I would like to p...|null|         etanol| null|1319395600|comment| 3146879| 3145330|       null|   null|   null|2011-10-23 14:46:40|
|                null|                null|                null|null|           null| null|1456640816|comment|11190

In [8]:
stopWords = set(stopwords.words('english'))

In [114]:
trad1 = open("/spring2021/project1/comparison/Charles Dickens - Cities.txt", 'r')

In [115]:
trad2 = open("/spring2021/project1/comparison/Mary Shelley - Frankenstein.txt", 'r')

In [116]:
trad3 = open("/spring2021/project1/comparison/Nathaniel Hawthorne - Scarlet.txt", 'r')

In [117]:
trad4 = open("/spring2021/project1/comparison/Herman Melville - Moby.txt", 'r')

In [118]:
trad5 = open("/spring2021/project1/comparison/Scott Fitzgerald - Gatsby.txt", 'r')

In [119]:
trad6 = open("/spring2021/project1/comparison/Jane Austen - Pride.txt", 'r')

In [120]:
trad3 = open("/spring2021/project1/comparison/Conan Doyle - Sherlock.txt", 'r')

In [121]:
def start_of_file(tradfile):
    tradfile.seek(0)
    i = -1
    while True:
        i += 1
        nxt = tradfile.readline()
        if nxt[0:3] == "***":
            tradfile.readline()
            i += 1
            print("The file is at the beginning. Line: " + str(i))
            break

In [130]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [139]:
def word_dict(tradfiles):
    for tradfile in tradfiles:
        start_of_file(tradfile)
        lines = tradfile.readlines()
        wordCnt = {'stop_words':0}
        for line in lines:
            rmRet = ' '.join(line.split('\n'))
            puncRm = ''.join([c for c in rmRet if c not in string.punctuation + '“”'])
            words = puncRm.split(' ')
            for word in words:
                word = word.lower()
                if word in stopWords:
                    wordCnt['stop_words'] += 1
                elif word not in list(wordCnt.keys()) and word != '':
                    wordCnt[word] = 1
                elif word != '':
                    wordCnt[word] += 1
    return wordCnt

In [140]:
filelist = [trad1, trad2, trad3, trad4, trad5, trad6]

In [141]:
word_counts = word_dict(filelist)

The file is at the beginning. Line: 22
The file is at the beginning. Line: 26
The file is at the beginning. Line: 21
The file is at the beginning. Line: 24
The file is at the beginning. Line: 25
The file is at the beginning. Line: 24


In [142]:
word_counts

{'stop_words': 67840,
 'illustrated': 1,
 'edition': 2,
 'title': 2,
 'may': 207,
 'viewed': 2,
 'ebook': 8,
 '42671': 1,
 'cover': 2,
 'pride': 46,
 'prejudice': 7,
 'jane': 260,
 'austen': 1,
 'contents': 10,
 'chapter': 122,
 '1': 4,
 '2': 4,
 '3': 5,
 '4': 5,
 '5': 3,
 '6': 3,
 '7': 2,
 '8': 2,
 '9': 2,
 '10': 2,
 '11': 2,
 '12': 2,
 '13': 2,
 '14': 2,
 '15': 2,
 '16': 2,
 '17': 2,
 '18': 2,
 '19': 2,
 '20': 3,
 '21': 2,
 '22': 2,
 '23': 2,
 '24': 2,
 '25': 2,
 '26': 2,
 '27': 2,
 '28': 2,
 '29': 2,
 '30': 3,
 '31': 2,
 '32': 2,
 '33': 2,
 '34': 2,
 '35': 2,
 '36': 2,
 '37': 2,
 '38': 2,
 '39': 2,
 '40': 2,
 '41': 2,
 '42': 2,
 '43': 2,
 '44': 2,
 '45': 2,
 '46': 2,
 '47': 2,
 '48': 2,
 '49': 2,
 '50': 3,
 '51': 2,
 '52': 2,
 '53': 2,
 '54': 2,
 '55': 2,
 '56': 2,
 '57': 2,
 '58': 2,
 '59': 2,
 '60': 3,
 '61': 2,
 'truth': 27,
 'universally': 3,
 'acknowledged': 20,
 'single': 12,
 'man': 142,
 'possession': 10,
 'good': 182,
 'fortune': 39,
 'must': 318,
 'want': 44,
 'wife': 44,


In [144]:
len(word_counts.values())

7064

In [3]:
blog = pd.read_csv('/spring2021/project1/blogtext.csv')

In [4]:
blog['length'] = blog['text'].map(lambda x: x.split(' ')).apply(len)

In [5]:
blog.head()

,id,gender,age,topic,sign,date,text,length
0,2059027,male,15,Student,Leo,"14,May,2004","Info has been found (+/- 100 pages,...",48
1,2059027,male,15,Student,Leo,"13,May,2004",These are the team members: Drewe...,84
2,2059027,male,15,Student,Leo,"12,May,2004",In het kader van kernfusie op aarde...,4363
3,2059027,male,15,Student,Leo,"12,May,2004",testing!!! testing!!!,24
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",Thanks to Yahoo!'s Toolbar I can ...,93


In [6]:
blog.describe()['length']

count    681284.000000
mean        233.000442
std         438.194357
min           5.000000
25%          61.000000
50%         140.000000
75%         290.000000
max      134557.000000
Name: length, dtype: float64

In [17]:
blog.loc[blog['length']==134557,'text']

529872           Excerpts from the HRW report:  [prologu...
Name: text, dtype: object

In [159]:
tradRDD = sc.parallelize((word_counts)

In [164]:
tradRDD.take(5)

KeyboardInterrupt: 

In [7]:
blogRDD = sc.parallelize(blog.text)

In [7]:
blogRDD.take(1)

['           Info has been found (+/- 100 pages, and 4.5 MB of .pdf files) Now i have to wait untill our team leader has processed it and learns html.         ']

In [9]:
def mapStopwords(x):
    rmPunc = ''.join([c for c in x if x not in string.punctuation])
    words =  rmPunc.split(' ')
    c = 0
    for word in words:
        if word in stopWords:
            c += 1
    return c

In [10]:
swRDD = blogRDD.map(mapStopwords)

In [11]:
swRDD.take(2)

[11, 2]

In [12]:
from operator import add